In [3]:
import os
os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_key
os.environ["SERPER_API_KEY"] = "caabb8481fd9568ce9e4f6149139098b7fd91f85"
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain.schema import Document as lcDocument
from langchain_pinecone import PineconeVectorStore
from langchain_community.utilities import GoogleSerperAPIWrapper
import requests
from bs4 import BeautifulSoup
from readability import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

In [31]:
embedding_model = HuggingFaceEmbeddings()

c:\1THUMUCDAIHOC\HOC_KY_7\PROJECT_AI\postpartum_psychology_sup_bot\.env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
db = PineconeVectorStore.from_documents(documents=[], embedding=embedding_model, index_name="docs-rag-chatbot", namespace="docs-link" )

In [55]:
def is_not_existed_url(url):
    results = db.similarity_search(query=url, namespace="docs-link", k=1)
    if results[0].page_content == url:
        return False

    return True

In [48]:
def store_links(links):
    if links:
        db = PineconeVectorStore.from_documents(documents=links, embedding=embedding_model, index_name="docs-rag-chatbot", namespace="docs-link" )

In [49]:
def add_url(results):    
    web_search_links = []
    for web in results['organic']:
        if is_not_existed_url(web['link']):
            link_doc = Document(
                page_content=web['link'],
                metadata={'source': web['title']})
            web_search_links.append(link_doc)
    print(web_search_links)
    store_links(web_search_links)

In [ ]:
class WebSearchDB():
    def __init__(self):
        self.embedding = HuggingFaceEmbeddings()
        self.db = self.build_db()
        self.text_slit = RecursiveCharacterTextSplitter(
                            separators = ["\n\n", "\n", " ", ".", ",", "\u200b", "\u3001", "\uff0e", "\u3002", ""],
                            chunk_size = 500,
                            chunk_overlap = 50,
                            length_function = len,
                            is_separator_regex = False)

    def build_db(self):
        index_name = "docs-rag-chatbot"
        namespace = "docs-link"
        try:
            results = self.db.similarity_search(query="test", namespace=namespace, k=1)
            if results:
                print(f"Namespace '{namespace}' đã tồn tại.")
                return self.db
        except Exception as e:
            print(f"Namespace '{namespace}' chưa tồn tại, sẽ tạo mới: {e}")
            
        dummy_texts = ["Initial text for new namespace"]
        db = PineconeVectorStore.from_texts(dummy_texts, index_name=index_name, embedding=self.embedding, namespace=namespace)
        return db
    
    def is_not_existed_url(self, url):
        results = self.db.similarity_search(query=url, namespace="docs-link", k=1)
        if results and results[0].page_content == url:
            return False
        return True
    
    def store_pinecone(self, docs, namespace):
        if docs:
            db = PineconeVectorStore.from_documents(documents=docs, embedding=self.embedding, index_name="docs-rag-chatbot", namespace=namespace)

    def store_web_content(self, web_links):
        documents = []
        if web_links:
            for link in web_links:
                content, title = self.read_content_from_web(link)
                if content:  # Check if content is valid
                    doc = lcDocument(page_content=content, metadata={"source": title})
                    chunks = self.text_slit.split_documents([doc]) 
                    cleaned_chunks = self.clean_data(chunks)
                
                    for i, chunk in enumerate(cleaned_chunks):
                        doc_chunk = lcDocument(
                            page_content=chunk.page_content,
                            metadata={"source": title, "chunk_id": i}
                        )
                        documents.append(doc_chunk)

            self.store_pinecone(documents, 'docs-store')

    def clean_data(self, chunks):
        cleaned_chunks = []
        for chunk in chunks:
            cleaned_content = chunk.page_content.replace("\n", " ")
            cleaned_content = re.sub(r'\s+', ' ', chunk.page_content)
            cleaned_content = cleaned_content.strip()
            cleaned_chunks.append(lcDocument(page_content=cleaned_content, metadata=chunk.metadata))
        return cleaned_chunks
        
    def read_content_from_web(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            doc = Document(response.content)
            html_content = doc.summary()
            title = doc.title()
            soup = BeautifulSoup(html_content, "html.parser")
            text_content = soup.get_text()
            return text_content, title
        return None, None

    def add_url(self, results):    
        links_document = []
        links = []
        for web in results['organic']:
            if self.is_not_existed_url(web['link']):
                link_doc = lcDocument(page_content=web['link'], metadata={'source': web['title']})
                links_document.append(link_doc)
                links.append(web['link'])
        self.store_pinecone(links_document, namespace='docs-link')
        self.store_web_content(links)

In [5]:
search = GoogleSerperAPIWrapper()
search.k = 2
results = search.results("Làm sao giảm stress")
results
#add_url(results)

{'searchParameters': {'q': 'Làm sao giảm stress',
  'gl': 'us',
  'hl': 'en',
  'type': 'search',
  'num': 2,
  'engine': 'google'},
 'organic': [{'title': '18 cách xả stress hiệu quả, giúp giảm bớt triệu chứng nhanh chóng',
   'link': 'https://tamanhhospital.vn/xa-stress/',
   'snippet': 'Mục lục · 1. Ghi nhận điều tích cực · 2. Kiểm soát cảm xúc · 3. Chế độ ăn uống lành mạnh · 4. Tập thể dục thường xuyên · 5. Ngừng sử dụng thuốc ...',
   'date': 'Oct 25, 2023',
   'sitelinks': [{'title': 'Stress ảnh hưởng tới cuộc...',
     'link': 'https://tamanhhospital.vn/xa-stress/#stress-anh-huong-toi-cuoc-song-nhu-the-nao'},
    {'title': 'Hướng dẫn cách xả stress...',
     'link': 'https://tamanhhospital.vn/xa-stress/#huong-dan-cach-xa-stress-hieu-qua-giup-giam-bot-trieu-chung-nhanh-chong'},
    {'title': 'Kết nối với mọi người',
     'link': 'https://tamanhhospital.vn/xa-stress/#12-ket-noi-voi-moi-nguoi'}],
   'position': 1},
  {'title': '“Bỏ túi” những cách giảm stress hiệu quả và đơn giản n

In [3]:


def read_content_from_web(url):
    response = requests.get(url)

    if response.status_code == 200:
        doc = Document(response.content)
        html_content = doc.summary()
        title = doc.title()
        soup = BeautifulSoup(html_content, "html.parser")
        text_content = soup.get_text()
        
        return text_content, title
    
    return None, None

In [4]:
for web in results["organic"]:
    text_content, title = read_content_from_web(web["link"])
    print(title)
    print(text_content)
    print("----------------------------------------------------")

Khắc phục tình trạng stress sau sinh

Nhiều mẹ bầu sau sinh thường cảm thấy mệt mỏi và căng thẳng. Stress sau sinh không chỉ ảnh hưởng đến tâm lý, cuộc sống của mẹ mà có thể ảnh hưởng đến cả em bé do không được mẹ chăm sóc tốt nhất. Cần phải khắc phục sớm tình trạng này để bảo vệ cả hai mẹ con.Stress sau sinh – Vấn đề chung của nhiều sản phụNhiều mẹ bầu, dù đã chuẩn bị kỹ càng tâm lý từ khi mang bầu nhưng sau khi sinh, những mệt mỏi, căng thẳng, tress là khó tránh khỏi. Nguyên nhân là do mẹ chưa quen hoặc chưa thể tự điều chỉnh khi phải đối diện với việc chăm sóc một đứa trẻ, nhất là với những người lần đầu làm mẹ.Việc chăm sóc trẻ sơ sinh không hề đơn giản. Mẹ luôn gặp phải những áp lực, muộn phiền khi trẻ quấy khóc, không bú hay khi trẻ bị ốm, sốt… Từ khi có con, cuộc sống của mẹ dường như thay đổi tất cả nên khó tránh khỏi tâm lý căng thẳng. Nếu bé ngoan, bú mẹ no rồi ngủ ngoan thì mẹ có nhiều thời gian để nghỉ ngơi, phục hồi sức khỏe nhưng những đứa bé không chịu ăn, chịu ngủ thì m